In [3]:
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

In [ ]:
model = gp.Model('Intento')
y = model.addVars([1,2], lb = 0, vtype = GRB.CONTINUOUS)
obj = np.array([-24,-28])
model.modelSense = GRB.MINIMIZE
model.setObjective( quicksum(y[a]*obj[a-1] for a in range(1,3)) )

lhs_ineq = np.array([[6,10],[8,5],[1,0],[0,1]])
rhs_ineq = np.array([2400,1600,500,100])

model.addMConstr (lhs_ineq, x = None, sense='<=', b = rhs_ineq, name="cons" )
model.optimize()
print("\nResultado:\n")
print(y,"\n")
print("Pi del modelo: \n")
print(model.pi)
print("Optimal objective\n", model.objval)

In [ ]:
#Ejemplo
#Ejemplo
c = np.array([100,150,0,0,0])
A = np.array([[1,1,1,0,0],[1,0,0,-1,0],[0,1,0,0,-1]])
b = np.array([120,40,20])

problemaMaestro(np.array([]), np.array([]), c, A, b)

#Rta debería ser: (40,20) y theta = -inf

In [ ]:
#Ejemplo caso 2:

E = np.array([[83.52, 180.48, 0,0,0,-520]])

c = np.array([100,150,0,0,0])
A = np.array([[1,1,1,0,0],[1,0,0,-1,0],[0,1,0,0,-1]])
b = np.array([120,40,20])


rta = problemaMaestro(np.array([]), E, c, A, b)
print(rta)

#Rta debería ser: (40,80) y  theta = -18299.2

In [ ]:
a = np.array([3,2,1])
b = np.append(a,np.array([4]))
b

In [ ]:
np.append(np.full(7,9),np.zeros(12))

In [ ]:
#Funciona: x in K2

k = 0
W= np.array([ [3,2,1,0,0,0,0,0],[2,5,0,1,0,0,0,0],[1,0,0,0,-1,0,0,0],[0,1,0,0,0,-1,0,0],[1,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,1] ])
q = np.array( [15,12,0,0,0,0,0,0] )

print(W)

#Itero sobre todas las posibles realizaciones de \zeta:
hact = np.array([0,0,4.8,6.4,6,8])
bact = hact 

model = gp.Model('Auxiliar')

vary = model.addVars(range(0,q.size), lb = 0, vtype = GRB.CONTINUOUS, name='vary')
vmas = model.addVars(range(0,W.shape[0]), lb = 0, vtype = GRB.CONTINUOUS, name = 'vmas')
vmenos = model.addVars(range(0,W.shape[0]), lb = 0, vtype = GRB.CONTINUOUS, name = 'vmenos')

print(model.getVars())

model.modelSense = GRB.MINIMIZE
model.setObjective( quicksum(vmas[a] + vmenos[a]  for a in range(0,W.shape[0])) )

Waux = np.hstack( (W, np.identity(W.shape[0]), -np.identity(W.shape[0])) )

model.addMConstr( Waux, x=None, sense='=', b = bact, name="constr" )

model.optimize()




In [ ]:
W= np.array([ [3,2,1,0,0,0,0,0],[2,5,0,1,0,0,0,0],[1,0,0,0,-1,0,0,0],[0,1,0,0,0,-1,0,0],[1,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,1] ])

print(W.shape[0])

Wact = np.hstack( (W, np.identity(W.shape[0]), -np.identity(W.shape[0])) )
Wact

In [4]:
# D es una matriz rx2 donde en la primera columna están los valores de D_l y en la segunda los de d_l
def problemaMaestro(D, E, c, A, b):
    model = gp.Model('Maestro')
    
    #Verifica que las entradas estén bien.
    if( D.any() and D.shape[1] != c.size+1 ):
        return('Error en las entradas. D debe ser r x (c.size)')
    
    if( E.any() and E.shape[1] != c.size+1 ):
        return('Error en las entradas. E debe ser s x (c.size)')
    
    if( E.any() == False and E.any() == False ): #Está funcionando.
        x = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS)
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(x[a]*c[a] for a in range(0,c.size)) )
        
        model.addMConstr (A, x = None, sense='=', b = b, name="constr" )
        model.optimize()
        rta = np.array([x[i].x for i in range(0,c.size)])
        return(rta, float('-Inf'), model.pi)
    
    elif( D.any() == False ): #Está funcionando.
        e = E[:,-1]
        E = np.hstack((E[:,:-1], np.ones((E.shape[0],1))))
        varx = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
        theta = model.addVars(1, lb = float('-Inf'), vtype = GRB.CONTINUOUS, name = 'theta')

        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(varx[a]*c[a] for a in range(0,c.size)) + theta[0] )

        list = [varx[i] for i in range(0,c.size)]
        model.addMConstr (A, x = list, sense='=', b = b, name="constr" )

        list.append(theta[0])
        model.addMConstr (E, x = list, sense='>=', b = e, name="constrE" ) 

        model.optimize()
        rta = np.array([varx[i].x for i in range(0,c.size)])
        return(rta, theta[0].x, model.pi)
    
    elif( E.any() == False ):
        d = D[:,-1]
        D = D[:,:-1] 

        x = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS)
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(x[a]*c[a] for a in range(0,c.size)) )
        
        model.addMConstr (A, x = None, sense='=', b = b, name="constr" )
        
        model.addMConstr (D, x = None, sense='>=', b = d )
        
        model.optimize()
        rta = np.array([x[i].x for i in range(0,c.size)])

        return(rta, float('-Inf'), model.pi)
    
    else: 
        e = E[:,-1]
        E = np.hstack((E[:,:-1], np.ones((E.shape[0],1))))
        d = D[:,-1]
        D = D[:,:-1] 
        
        varx = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
        theta = model.addVars(1, lb = float('-Inf'), vtype = GRB.CONTINUOUS, name = 'theta')
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(varx[a]*c[a] for a in range(0,c.size)) + theta[0] )
        list = [varx[i] for i in range(0,c.size)]
        model.addMConstr (A, x = list, sense='=', b = b, name="constr" )
        model.addMConstr (D, x = list, sense='>=', b = d, name="constrD" )

        list.append(theta[0])
        model.addMConstr (E, x = list, sense='>=', b = e, name="constrE" ) 
        
        model.optimize()
        rta = np.array([varx[i].x for i in range(0,c.size)])
        return(rta, theta[0].x, model.pi)

# Intento

In [5]:
#Retorna true si x está en K2, false de lo contrario.

zetaPosibles = [(500,100,-24,-28,0.4), (300,300,-28,-32,0.6)]

c = np.array([100,150,0,0,0])
b = np.array([120,40,20])
q = np.zeros(6)
# T es 4x5
T = np.array([[-60,0,0,0,0],[0,-80,0,0,0],[0,0,0,0,0],[0,0,0,0,0]])
W = np.array([[6,10,1,0,0,0],[8,5,0,1,0,0],[1,0,0,0,1,0],[0,1,0,0,0,1]])

A = np.array([[1,1,1,0,0],[1,0,0,-1,0],[0,1,0,0,-1]])

def xInK2(x):
    k = 0
    #Itero sobre todas las posibles realizaciones de \zeta:
    for (d1,d2,q1,q2,p) in zetaPosibles:
        hact = np.array([d1,d2,q1,q2])
        bact = hact - (T @ x)

        model = gp.Model('Auxiliar')

        vary = model.addVars(range(0,q.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
        vmas = model.addVars(range(0,W.shape[0]), lb = 0, vtype = GRB.CONTINUOUS, name = 'vmas')
        vmenos = model.addVars(range(0,W.shape[0]), lb = 0, vtype = GRB.CONTINUOUS, name = 'vmenos')

        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(vmas[a] + vmenos[a]  for a in range(0,W.shape[0])) )

        Waux = np.hstack( (W, np.identity(W.shape[0]), -np.identity(W.shape[0])) ) #Bien

        lista = [vary[i] for i in range(0,q.size)]
        lista.extend( [vmas[i] for i in range(0,W.shape[0])] )      
        lista.extend( [vmenos[i] for i in range(0,W.shape[0])] )
        print(lista)
        model.addMConstr( Waux, x=lista, sense='=', b = bact, name="constr" )

        model.optimize()
        print(model.objval)
        if( model.objval > 0 ):
            print(hact)
            return([model.pi, hact] )
            break

    return np.array([])

In [ ]:
xInK2(np.array([40,20,0,0,0]))

In [35]:

def Lshaped():
    s= 0; r=0; v=0; K=5000
    #En nuestro caso, A, b, c, W, T y q son constantes. 
    #Lo único que cambia a partir de las realizaciones de \zeta es h_k
    D = np.array([]); E = np.array([])
    parar = False
    
    while(parar == False):
        v += 1
        print('##################################################################################\n', v ,'\n#################################################################################################' )
        (xv,thetav,pi) = problemaMaestro(D, E, c, A, b)

        pi = np.zeros((W.shape[0],2)) #Cambiar 2 por K 
        pih = np.zeros(2)
        k = 0
        #Itero sobre todas las posibles realizaciones de \zeta:
        for (d1,d2,q1,q2,p) in zetaPosibles:
            hact = np.array([0,0,d1,d2])
            bact = hact - (T @ xv)
            #NOTA: en el anterior para calcular e_s usaba el mismo hact
            qk = np.array([q1,q2,0,0,0,0])
            (y,t,piact) = problemaMaestro(np.array([]),np.array([]),qk,W,bact) 
            pi[:,k] = piact #Revisar
            pih[k] = p * np.dot(piact,hact)
            k+=1
        print('\n-------pih',pih)

        #Calculo E
        E_s =  np.zeros(c.size) 
        matriz = np.vstack((0.4*pi[:,0] @ T, 0.6*pi[:,1] @ T))

        for i in range(0,5): #Cambiar esto! 
            E_s[i] = np.sum( matriz[:,i] ) 

        #Calculo e
        e_s =  np.sum( pih )
        
        wv = e_s - np.dot(E_s,xv) #CAMBIAR ESTO POR MENOS
        
        
        print('\n $$$$$$ e_s $$$$$$$$\n',e_s)
        print('\n $$$$$$ E_s $$$$$$$$\n',E_s)
        print('\n $$$$$$ xv $$$$$$$$\n',xv)
        print('\n $$$$$$ E_s xv $$$$$$$$\n', np.dot(E_s,xv))
        print('\n $$$$$$ wv $$$$$$$$\n',wv)
        print('\n $$$$$$ thetav $$$$$$$$\n',thetav)

        parar = ( np.all(wv <= thetav) )
        
        if(parar == False):
            s = s+1
            if E.any() == False:
                E = np.array([np.hstack( (E_s, e_s) )])
            else:
                E = np.vstack( (E, np.hstack( (E_s, e_s) )) )

        print( xv )

In [36]:
Lshaped()
#hact = np.array([0,0,500,100])
#bact = hact - (T @ np.array( [40,20,60,0,0] ))

#problemaMaestro(np.array([]),np.array([]),np.array([-24,-28,0,0,0,0]),W,bact) 

##################################################################################
 1 
#################################################################################################
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3 rows, 5 columns and 7 nonzeros
Model fingerprint: 0xad331c12
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+02]
Presolve removed 3 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.000000000e+03
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimi

Optimize a model with 6 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x6eb37202
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Presolve removed 3 rows and 3 columns
Presolve time: 0.02s
Presolved: 3 rows, 3 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -9.7040000e+03   5.432000e+02   0.000000e+00      0s
       3   -8.8950000e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective -8.895000000e+02
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 6 columns and 10 nonzeros
Model fingerprint: 0xfe7e1464
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+03]
Presolve